In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from datetime import datetime
from pathlib import Path
import pickle
import sys

import matplotlib.pyplot as plt
from pymatgen.core.structure import Molecule

Custom plotting code... ignore this if you don't know what it is, it should gracefully do nothing if you don't have the `MPLAdjutant` class. 

In [ ]:
sys.path.append(str(Path.home() / Path("local")))
class NullClass:
    def do_nothing(*args, **kwargs):
        pass
    def add_colorbar(self, im, **kwargs):
        return plt.colorbar(im)
    def __getattr__(self, _):
        return self.do_nothing
try:
    from mpl_utils import MPLAdjutant
    adj = MPLAdjutant()
    adj.set_defaults()
except ImportError:
    adj = NullClass()

Append the `home` path of this project.

In [ ]:
sys.path.append(str(Path.cwd().parent))

# Principal component analysis: verifying the data

In this notebook, we perform basic analysis and Principal Component Analysis to validate that our dataset look sensible.

In [ ]:
from sklearn.decomposition import PCA

Load in the previously processed `pickle` data. These contain the features (ACSF) and targets (spectra) on a common grid for each of the three absorbing atom types of interest: C, N and O.

In [ ]:
O_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-O.pkl", "rb"))
N_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-N.pkl", "rb"))
C_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-C.pkl", "rb"))

Print some diagonstics...

In [ ]:
print(f"O database ACSF & spectrum shapes: {O_data['x'].shape} & {O_data['y'].shape}")
print(f"N database ACSF & spectrum shapes: {N_data['x'].shape} & {N_data['y'].shape}")
print(f"C database ACSF & spectrum shapes: {C_data['x'].shape} & {C_data['y'].shape}")

Put everything in a dictionary for simplicity.

In [ ]:
data = {"O": O_data, "N": N_data, "C": C_data}

## Execute and plot

In [ ]:
pca_results = dict()
for key, dat in data.items():
    pca_acsf = PCA(2)
    w_acsf = pca_acsf.fit_transform(dat["x"])
    pca_spectra = PCA(2)
    w_spectra = pca_spectra.fit_transform(dat["y"])
    pca_results[key] = {
        "pca_ACSF": pca_acsf,
        "pca_spectra": pca_spectra,
        "w_ACSF": w_acsf,
        "w_spectra": w_spectra
    }

In [ ]:
pca_results.keys()

In [ ]:
for key, dat in pca_results.items():
    pca_ACSF = dat["pca_ACSF"]
    pca_spectra = dat["pca_spectra"]
    print(f"Key={key}, ACSF var ratios={pca_ACSF.explained_variance_ratio_}, spectra var ratios={pca_spectra.explained_variance_ratio_}")

We make a production-quality plot of the data here:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(6, 2))

N = None
key_ordering = ["C", "N", "O"]

for key, ax in zip(key_ordering, axs):
    w_acsf = pca_results[key]["w_ACSF"]
    w_spectra = pca_results[key]["w_spectra"]
    ax.scatter(w_acsf[:N, 0], w_acsf[:N, 1], alpha=0.02, c=w_spectra[:N, 0], s=1, cmap="rainbow", rasterized=True)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_linewidth(0.5)
    ax.spines["bottom"].set_linewidth(0.5)
    ax.tick_params(left=False, bottom=False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_title(key)

axs[0].set_ylabel(r"ACSF $z_2$")
axs[1].set_xlabel(r"ACSF $z_1$")
    
plt.savefig("qm9_pca.pdf", bbox_inches="tight", dpi=300)

# Train/validation-test split

In [ ]:
from xas_nne.qm9 import split_qm9_data_by_number_of_absorbers

## Option 1: train on molecules with at most two absorbers

In [ ]:
for CENTRAL_ATOM, max_training_absorbers in zip(["C", "N", "O"], [7, 2, 2]):
    parsed_data = split_qm9_data_by_number_of_absorbers(data[CENTRAL_ATOM], absorber=CENTRAL_ATOM, max_training_absorbers=max_training_absorbers)
    parsed_data["metadata"] = {"max_training_absorbers": max_training_absorbers}
    now = datetime.now().strftime("%y%m%d")
    fname = f"../data/qm9/ml_ready/XANES-{now}-ACSF-{CENTRAL_ATOM}-MAX_TRAINING_ABSORBERS-{max_training_absorbers}.pkl"
    print(fname)
    pickle.dump(parsed_data, open(fname, "wb"), protocol=pickle.HIGHEST_PROTOCOL)

# Do certain structural motifs cluster?